# Lattice v0.3 — Phase 2 Demo

This notebook demonstrates the full Lattice pipeline API with a tiny dataset (3 rows, ~$0.001 total cost).

**What you'll see:**
- `Pipeline.run(df)` / `pipeline.run_async(df)` — the primary API
- Inline field specs on LLMStep
- `PipelineResult` with cost tracking and error reporting
- Multi-step pipelines with FunctionStep + LLMStep
- Per-row error handling (one row fails, others succeed)
- Progress bars via tqdm

> **Note:** Jupyter runs its own async event loop, so we use `await pipeline.run_async(df)` here.
> In scripts, use `pipeline.run(df)` (sync wrapper) instead.

In [ ]:
import pandas as pd
from lattice import Pipeline, LLMStep, FunctionStep, EnrichmentConfig

## 1. Simple: One LLM step with inline fields

The simplest possible usage — 3 rows, 2 fields, one API call per row.

Uses `gpt-4.1-nano` by default ($0.10/1M input, $0.40/1M output) — this cell costs ~$0.001.

In [ ]:
df = pd.DataFrame({
    "company": ["Stripe", "Notion", "Figma"],
    "description": [
        "Online payment processing for internet businesses",
        "All-in-one workspace for notes, docs, and project management",
        "Collaborative interface design tool for teams",
    ],
})
df

In [ ]:
pipeline = Pipeline([
    LLMStep("analyze", fields={
        "category": "Classify into one of: Fintech, Productivity, Design, Other",
        "target_market": "Describe the primary target market in 10 words or less",
    })
])

result = await pipeline.run_async(df)
result.data

In [ ]:
# Cost and error summary
print(f"Success rate: {result.success_rate:.0%}")
print(f"Errors: {len(result.errors)}")
print(f"Total tokens: {result.cost.total_tokens}")
print(f"\nPer-step breakdown:")
for step_name, usage in result.cost.steps.items():
    print(f"  {step_name}: {usage.total_tokens} tokens ({usage.rows_processed} rows, model={usage.model})")

## 2. Multi-step pipeline with dependencies

FunctionStep generates context → LLMStep uses it. Shows dependency routing and `__` internal fields.

In [ ]:
def generate_context(ctx):
    """Simulate an API lookup — in production this could be a web search, CRM call, etc."""
    fake_data = {
        "Stripe": "Founded 2010. $95B valuation. 8000+ employees. Competes with Adyen, Square.",
        "Notion": "Founded 2013. $10B valuation. 500+ employees. Competes with Confluence, Coda.",
        "Figma": "Founded 2012. Acquired by Adobe for $20B (cancelled). Competes with Sketch, Canva.",
    }
    company = ctx.row["company"]
    return {"__context": fake_data.get(company, "No data available")}


pipeline = Pipeline([
    FunctionStep("lookup", fn=generate_context, fields=["__context"]),
    LLMStep("synthesize", fields={
        "competitive_position": "Rate as Leader/Challenger/Niche based on the context",
        "investment_thesis": "One-sentence investment thesis using context and description",
    }, depends_on=["lookup"]),
])

result = await pipeline.run_async(df)
result.data

In [ ]:
# __context is NOT in the output — internal fields are filtered
print("Columns:", list(result.data.columns))
assert "__context" not in result.data.columns

## 3. Per-row error handling

One row throws an error — the other rows still complete. No crash.

In [ ]:
def flaky_lookup(ctx):
    """Simulates an API that fails for one company."""
    if ctx.row["company"] == "Notion":
        raise ConnectionError("API timeout for Notion")
    return {"status": f"{ctx.row['company']} OK"}


pipeline = Pipeline([
    FunctionStep("check", fn=flaky_lookup, fields=["status"]),
])

result = await pipeline.run_async(df)

print(f"Success rate: {result.success_rate:.0%}")
print(f"Errors: {len(result.errors)}")
for err in result.errors:
    print(f"  Row {err.row_index} ({df.iloc[err.row_index]['company']}): {err.error_type} — {err.error}")

result.data[["company", "status"]]

## 4. Custom config

Control concurrency, retries, progress bars, and error mode.

In [ ]:
config = EnrichmentConfig(
    max_workers=2,           # concurrent rows
    temperature=0.1,         # low for deterministic output
    max_retries=2,           # API error retries
    enable_progress_bar=True,
)

pipeline = Pipeline([
    LLMStep("tag", fields={
        "keywords": "List 3 keywords as a comma-separated string",
    })
])

result = await pipeline.run_async(df, config=config)
result.data[["company", "keywords"]]

## 5. Using a different provider (base_url)

Any OpenAI-compatible API works via `base_url`. Uncomment one of these to try:

```python
# Ollama (local)
LLMStep("analyze", fields={...}, model="llama3", base_url="http://localhost:11434/v1")

# Groq
LLMStep("analyze", fields={...}, model="llama-3.3-70b-versatile", base_url="https://api.groq.com/openai/v1", api_key="gsk_...")

# Anthropic (requires: pip install lattice[anthropic])
from lattice.providers import AnthropicClient
LLMStep("analyze", fields={...}, model="claude-sonnet-4-5-20250929", client=AnthropicClient())

# Google (requires: pip install lattice[google])
from lattice.providers import GoogleClient
LLMStep("analyze", fields={...}, model="gemini-2.5-flash", client=GoogleClient())
```

## Summary

| Feature | How |
|---------|-----|
| Run pipeline | `pipeline.run(df)` → `PipelineResult` |
| Inline fields | `LLMStep("name", fields={"field": "prompt"})` |
| Cost tracking | `result.cost.total_tokens`, `result.cost.steps` |
| Error handling | `result.errors`, `result.success_rate` |
| Multi-step | `depends_on=["step_name"]` |
| Internal fields | `__` prefix — filtered from output |
| Providers | `base_url=`, `client=AnthropicClient()` |
| Config | `EnrichmentConfig(max_workers=10, temperature=0.1)` |